# 基于朴素贝叶斯的新闻分类

In [39]:
import numpy as np
import pandas as pd
import jieba

# 1.读取数据集

数据集一共有五千条新闻

In [40]:
df_news = pd.read_table("D:\刘航\SampleCode\data\data.txt",names=['label','theme','URL','content'],encoding='utf-8')
df_news = df_news.dropna()#删除缺失值
#df_news.tail()
df_news.shape

(5000, 4)

从上面不难看出我们的数据一共有五千篇文章和四个特征，而我们主要用到的就是文章内容，我们要根据文章的内容来判断文章的类别，接下来就是提取我们要用到的数据

In [41]:
content = df_news['content'].values.tolist()
#content[100]

# 2.分词，删除停用词

In [42]:
content_S = []
for text in content:
    cut_content = jieba.lcut(text)
    if len(cut_content)>1 and cut_content != '\r\n':
        content_S.append(cut_content)
#content_S[100]

接下来删除停用词

In [43]:
stopwords = pd.read_csv("D:\刘航\SampleCode\stopwords.txt",sep='t',quoting=3,names=['stopwords'],encoding='utf-8')
#stopwords.head()

In [44]:
#删除停用词方法
def drop_stopwords(contents,stopwords):
    content_clearn = []
    for line in contents:
        line_clearn = []
        for word in line:
            if word in stopwords:
                continue
            line_clearn.append(word)
        content_clearn.append(line_clearn)
    return content_clearn

stopwords = stopwords.stopwords.values.tolist()
content_clearn = drop_stopwords(content_S,stopwords)

df_content = pd.DataFrame({'content_clearn':content_clearn})
df_content.head()

,content_clearn
0,"[经销商, 电话, 试驾, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, 路, 号, 转, ..."
1,"[呼叫, 热线, 服务, 邮箱, ｋ, ｆ, ｐ, ｅ, ｏ, ｐ, ｌ, ｅ, ｄ, ａ,..."
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 二月, 公布, 最新, Ｍ, Ｉ, Ｎ, Ｉ, 新, 概念..."
3,"[清仓, 甩卖, 一汽, 夏利, Ｎ, 威志, Ｖ, 低至, 万, 启新, 中国, 一汽, ..."
4,"[日内瓦, 车展, 见到, 高尔夫, 家族, 新, 成员, 高尔夫, 敞篷版, 款, 全新,..."


# 3.划分训练集和测试集

这里我们主要用到的数据是我们做好处理的新闻和新闻原来的标签

In [45]:
df_train = pd.DataFrame({'content_clearn':content_clearn,'label':df_news['label']})
df_train.tail()

,content_clearn,label
4995,"[天气, 炎热, 补水, 变得, 美国, 跑步, 世界, 杂志, 报道, 喝水, 身体, 补...",时尚
4996,"[不想, 说, 话, 刺激, 说, 做, 只能, 走, 离开, 伤心地, 想起, 一句, 话...",时尚
4997,"[岁, 刘晓庆, 最新, 嫩照, Ｏ, 衷, 诘, 牧跸, 庆, 看不出, 岁, 秒杀, 刘...",时尚
4998,"[导语, 做, 爸爸, 一种, 幸福, 无论是, 领养, 亲生, 更何况, 影视剧, 中, ...",时尚
4999,"[全球, 最美, 女人, 合成图, 国, 整形外科, 教授, 李承哲, 国际, 学术, 杂志...",时尚


数据集标签制作

先查看数据集一共有一个标签，然后为标签命名，之后构建映射，更改掉对应的标签

In [46]:
df_train.label.unique()

array(['汽车', '财经', '科技', '健康', '体育', '教育', '文化', '军事', '娱乐', '时尚'],
      dtype=object)

In [47]:
label_mapping = {'汽车':1, '财经':2, '科技':3, '健康':4, '体育':5, '教育':6, '文化':7, '军事':8, '娱乐':9, '时尚':0}
df_train['label'] = df_train['label'].map(label_mapping)#构建一个映射方法
df_train.head()

,content_clearn,label
0,"[经销商, 电话, 试驾, 订车, Ｕ, 憬, 杭州, 滨江区, 江陵, 路, 号, 转, ...",1
1,"[呼叫, 热线, 服务, 邮箱, ｋ, ｆ, ｐ, ｅ, ｏ, ｐ, ｌ, ｅ, ｄ, ａ,...",1
2,"[Ｍ, Ｉ, Ｎ, Ｉ, 品牌, 二月, 公布, 最新, Ｍ, Ｉ, Ｎ, Ｉ, 新, 概念...",1
3,"[清仓, 甩卖, 一汽, 夏利, Ｎ, 威志, Ｖ, 低至, 万, 启新, 中国, 一汽, ...",1
4,"[日内瓦, 车展, 见到, 高尔夫, 家族, 新, 成员, 高尔夫, 敞篷版, 款, 全新,...",1


In [48]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_train['content_clearn'].values,df_train['label'].values,random_state=1)

#x_train.shape

# 词袋模型的构建



In [62]:
#首先将训练集中的所有词都存在一起
words = []
for line_index in range(len(x_train)):
    try:
        words.append(" ".join(x_train[line_index]))
    except:
        print(line_index)
#print(len(words))

制作词袋特征模型

In [70]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer='word', max_features=4000 ,lowercase=False)
feature = vec.fit_transform(words)
feature.shape

(3750, 4000)

导入贝叶斯模型进行训练

In [64]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(feature, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
#查看在训练集上的测试结果
classifier.score(feature,y_train)

0.8674666666666667

In [72]:
#查看测试集上的测试结果
test_words = []
for line_index in range(len(x_test)):
    try:
        test_words.append(" ".join(x_test[line_index]))
    except:
        print(line_index)
#len(test_words)

In [71]:
classifier.score(vec.transform(test_words),y_test)

0.804

利用TF-IDF进行训练

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectotizer = TfidfVectorizer(analyzer='word', max_features=4000,lowercase=False )
vectotizer.fit(words)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=4000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [74]:
classifier.fit(vectotizer.transform(words),y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [75]:
classifier.score(vectotizer.transform(words),y_train)

0.8794666666666666

In [76]:
classifier.score(vectotizer.transform(test_words),y_test)

0.8152